In [2]:
from collections import deque
import numpy as np
import argparse
import cv2
import time

In [9]:
# Setup color range, and deque for live and drawing
# lower=(29, 86, 250)
# upper=(64, 255, 255)

lower=(29, 80, 250)
upper=(255, 255, 255)

pts_draw=deque()
draw=False

In [10]:
# Initialize white board
whiteboard=np.ones([480,640,3])

# Open camera image

cap = cv2.VideoCapture(0)
time.sleep(2.0)

if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
        
    # Resize to half to reduce computation
#     frame=cv2.resize(frame,None,fx=1/2, fy=1/2, interpolation = cv2.INTER_CUBIC)
    # Apply Gaussian blur
    frame_blur=cv2.GaussianBlur(frame, (11, 11), 0)
    # Convert to HSV
    frame_hsv=cv2.cvtColor(frame_blur, cv2.COLOR_BGR2HSV)
    
    # Find mask containing target HSV values:
    mask=cv2.inRange(frame_hsv, lower, upper)
    
    # Extract contour of mask
    cnts, heirarchy = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)
    
    # break if no contours found
    while not cnts:
        print("Can't find selected range!!")
        break
    
    # Select largest contour if multiple are found:
    cnt = max(cnts, key=cv2.contourArea)
    
    # Find center of selected cnt
    ((x, y), radius) = cv2.minEnclosingCircle(cnt)
    M=cv2.moments(cnt)
    center=(int(M["m10"] / M["m00"]+pow(10,-8)), int(M["m01"] / M["m00"]+pow(10,-8)))

    # Draw a small circle=Dot in place of center of object
    cv2.circle(frame, (int(x), int(y)), 2, (255,0,0), -1)
    
    pts_draw.appendleft(center)
    
    k = cv2.waitKey(1) & 0xFF
    if k==ord('d'):
        draw=not(draw)
    
    if draw==True and x<480 and y<640:
        
        whiteboard.itemset((int(x),int(y),2),100)
        for i in range(1, len(pts_draw)):
            cv2.line(whiteboard, pts_draw[i - 1], pts_draw[i], (0, 0, 255), 5)


    cv2.imshow('Tracker', frame)
    cv2.imshow('Whiteboard', whiteboard)
#     pts_draw.appendleft(center)
    

    
    if cv2.waitKey(1) == ord('q'):
        break
    
        
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()        

Can't find selected range!!


ValueError: max() arg is an empty sequence

In [11]:
cap.release()
cv2.destroyAllWindows()  